# Ultimate Challenge 
# Part 2 

In [33]:
#Import necessary libraries
import pandas as pd
import numpy as np

# Plotting modules
import matplotlib.pyplot as plt
import seaborn as sns

# Analysing datetime
from datetime import datetime as dt
from datetime import timedelta

# File system manangement
import os,sys
from datetime import datetime
# Suppress warnings 
import warnings
warnings.filterwarnings('ignore')

#Interactive Shell
from IPython.core.interactiveshell import InteractiveShell  
InteractiveShell.ast_node_interactivity = "all"

#Pandas profiling
from pandas_profiling import ProfileReport

import missingno as msno
import re 

%matplotlib inline

In [34]:
#SKLEARN 

#MODEL_SELECTION
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

from sklearn.tree import DecisionTreeClassifier

#LINEAR_MODEL
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso, Ridge


#ENSEMBLE 
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import BaggingClassifier

#SVM
from sklearn.svm import SVC

from sklearn.neighbors import KNeighborsClassifier

from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB

#METRICS
from sklearn.metrics import r2_score 
from sklearn.metrics import accuracy_score
from sklearn.metrics import make_scorer
from sklearn.metrics import balanced_accuracy_score, precision_score
from sklearn.metrics import roc_curve, roc_auc_score, auc
from sklearn.metrics import explained_variance_score
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.metrics import classification_report, confusion_matrix

# PREPROCESSING
from sklearn.preprocessing import StandardScaler, OneHotEncoder,LabelEncoder

#PIPELINE 
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline as mpl



#### Load Json Data

In [35]:
# Import Json file and read the dataset
path = 'data2.json'
df = pd.read_json(path) #parse_dates=True
df.head()

,city,trips_in_first_30_days,signup_date,avg_rating_of_driver,avg_surge,last_trip_date,phone,surge_pct,ultimate_black_user,weekday_pct,avg_dist,avg_rating_by_driver
0,King's Landing,4,2014-01-25,4.7,1.10,2014-06-17,iPhone,15.4,True,46.2,3.67,5.0
1,Astapor,0,2014-01-29,5.0,1.00,2014-05-05,Android,0.0,False,50.0,8.26,5.0
2,Astapor,3,2014-01-06,4.3,1.00,2014-01-07,iPhone,0.0,False,100.0,0.77,5.0
3,King's Landing,9,2014-01-10,4.6,1.14,2014-06-29,iPhone,20.0,True,80.0,2.36,4.9
4,Winterfell,14,2014-01-27,4.4,1.19,2014-03-15,Android,11.8,False,82.4,3.13,4.9


In [36]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 12 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   city                    50000 non-null  object 
 1   trips_in_first_30_days  50000 non-null  int64  
 2   signup_date             50000 non-null  object 
 3   avg_rating_of_driver    41878 non-null  float64
 4   avg_surge               50000 non-null  float64
 5   last_trip_date          50000 non-null  object 
 6   phone                   49604 non-null  object 
 7   surge_pct               50000 non-null  float64
 8   ultimate_black_user     50000 non-null  bool   
 9   weekday_pct             50000 non-null  float64
 10  avg_dist                50000 non-null  float64
 11  avg_rating_by_driver    49799 non-null  float64
dtypes: bool(1), float64(6), int64(1), object(4)
memory usage: 4.2+ MB


In [37]:
df.describe(include='all').T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
city,50000,3,Winterfell,23336,NaN,NaN,NaN,NaN,NaN,NaN,NaN
trips_in_first_30_days,50000,NaN,NaN,NaN,2.2782,3.79268,0,0,1,3,125
signup_date,50000,31,2014-01-18,2948,NaN,NaN,NaN,NaN,NaN,NaN,NaN
avg_rating_of_driver,41878,NaN,NaN,NaN,4.60156,0.617338,1,4.3,4.9,5,5
avg_surge,50000,NaN,NaN,NaN,1.07476,0.222336,1,1,1,1.05,8
last_trip_date,50000,182,2014-06-29,2036,NaN,NaN,NaN,NaN,NaN,NaN,NaN
phone,49604,2,iPhone,34582,NaN,NaN,NaN,NaN,NaN,NaN,NaN
surge_pct,50000,NaN,NaN,NaN,8.84954,19.9588,0,0,0,8.6,100
ultimate_black_user,50000,2,False,31146,NaN,NaN,NaN,NaN,NaN,NaN,NaN
weekday_pct,50000,NaN,NaN,NaN,60.9261,37.0815,0,33.3,66.7,100,100


In [38]:
df.describe(include='all').T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
city,50000,3,Winterfell,23336,NaN,NaN,NaN,NaN,NaN,NaN,NaN
trips_in_first_30_days,50000,NaN,NaN,NaN,2.2782,3.79268,0,0,1,3,125
signup_date,50000,31,2014-01-18,2948,NaN,NaN,NaN,NaN,NaN,NaN,NaN
avg_rating_of_driver,41878,NaN,NaN,NaN,4.60156,0.617338,1,4.3,4.9,5,5
avg_surge,50000,NaN,NaN,NaN,1.07476,0.222336,1,1,1,1.05,8
last_trip_date,50000,182,2014-06-29,2036,NaN,NaN,NaN,NaN,NaN,NaN,NaN
phone,49604,2,iPhone,34582,NaN,NaN,NaN,NaN,NaN,NaN,NaN
surge_pct,50000,NaN,NaN,NaN,8.84954,19.9588,0,0,0,8.6,100
ultimate_black_user,50000,2,False,31146,NaN,NaN,NaN,NaN,NaN,NaN,NaN
weekday_pct,50000,NaN,NaN,NaN,60.9261,37.0815,0,33.3,66.7,100,100


___

### Check for missing values

In [39]:
df.isnull().values.any()

True

In [40]:
df.isna().sum()

city                         0
trips_in_first_30_days       0
signup_date                  0
avg_rating_of_driver      8122
avg_surge                    0
last_trip_date               0
phone                      396
surge_pct                    0
ultimate_black_user          0
weekday_pct                  0
avg_dist                     0
avg_rating_by_driver       201
dtype: int64

In [41]:
df = df.fillna(axis=0,method='ffill')

In [42]:
df.isna().sum()

city                      0
trips_in_first_30_days    0
signup_date               0
avg_rating_of_driver      0
avg_surge                 0
last_trip_date            0
phone                     0
surge_pct                 0
ultimate_black_user       0
weekday_pct               0
avg_dist                  0
avg_rating_by_driver      0
dtype: int64

### Convert types

In [43]:
df['signup_date'] = df['signup_date'].astype('datetime64[ns]')
df['last_trip_date'] = df['last_trip_date'].astype('datetime64[ns]')
df['ultimate_black_user'] = df['ultimate_black_user'].astype('object')


In [44]:
df['active_users'] = df['last_trip_date'].apply(lambda x: 1 if x >= pd.Timestamp('2014-06-01', tz=None) else 0)

In [45]:
df['active_users'].value_counts()

0    31196
1    18804
Name: active_users, dtype: int64

In [70]:
df['active_users'] = df['active_users'].astype('float')

In [73]:
df['trips_in_first_30_days'] = df['trips_in_first_30_days'].astype('float')

___

## Preprocessing

In [74]:
df.select_dtypes(include=[object])

,city,phone,ultimate_black_user
0,King's Landing,iPhone,True
1,Astapor,Android,False
2,Astapor,iPhone,False
3,King's Landing,iPhone,True
4,Winterfell,Android,False
...,...,...,...
49995,King's Landing,iPhone,False
49996,Astapor,iPhone,False
49997,Winterfell,Android,True
49998,Astapor,iPhone,False


### ONE HOT ENCODER

In [75]:
categorical_features = ['city','phone','ultimate_black_user']

In [76]:
df_le1 = df.loc[:, categorical_features]

In [77]:
le = LabelEncoder()

df_le1['city'] = le.fit_transform(df_le1['city'].astype(str))
df_le1['phone'] = le.fit_transform(df_le1['phone'].astype(str))
df_le1['ultimate_black_user'] = le.fit_transform(df_le1['ultimate_black_user'].astype(str))

dropped_cols = df.drop(columns=df_le1)

df_le = pd.concat([dropped_cols,df_le1],axis=1)

In [78]:
df_le

,trips_in_first_30_days,signup_date,avg_rating_of_driver,avg_surge,last_trip_date,surge_pct,weekday_pct,avg_dist,avg_rating_by_driver,active_users,city,phone,ultimate_black_user
0,4.0,2014-01-25,4.7,1.10,2014-06-17,15.4,46.2,3.67,5.0,1.0,1,1,1
1,0.0,2014-01-29,5.0,1.00,2014-05-05,0.0,50.0,8.26,5.0,0.0,0,0,0
2,3.0,2014-01-06,4.3,1.00,2014-01-07,0.0,100.0,0.77,5.0,0.0,0,1,0
3,9.0,2014-01-10,4.6,1.14,2014-06-29,20.0,80.0,2.36,4.9,1.0,1,1,1
4,14.0,2014-01-27,4.4,1.19,2014-03-15,11.8,82.4,3.13,4.9,0.0,2,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,0.0,2014-01-25,5.0,1.00,2014-06-05,0.0,100.0,5.63,4.2,1.0,1,1,0
49996,1.0,2014-01-24,5.0,1.00,2014-01-25,0.0,0.0,0.00,4.0,0.0,0,1,0
49997,0.0,2014-01-31,5.0,1.00,2014-05-22,0.0,100.0,3.86,5.0,0.0,2,0,1
49998,2.0,2014-01-14,3.0,1.00,2014-01-15,0.0,100.0,4.58,3.5,0.0,0,1,0


In [79]:
ohe = OneHotEncoder(sparse=True, drop='first', handle_unknown='error')


encoded = pd.DataFrame(ohe.fit_transform(df_le[categorical_features]).toarray(), \
                                columns=ohe.get_feature_names(categorical_features))

dropped_cols = df.drop(columns=categorical_features)


#df_ohe = data_hot_encoded_drop.join(data_hot_encoded)
df_ohe = pd.concat([dropped_cols,encoded],axis=1)

df_ohe

,trips_in_first_30_days,signup_date,avg_rating_of_driver,avg_surge,last_trip_date,surge_pct,weekday_pct,avg_dist,avg_rating_by_driver,active_users,city_1,city_2,phone_1,ultimate_black_user_1
0,4.0,2014-01-25,4.7,1.10,2014-06-17,15.4,46.2,3.67,5.0,1.0,1.0,0.0,1.0,1.0
1,0.0,2014-01-29,5.0,1.00,2014-05-05,0.0,50.0,8.26,5.0,0.0,0.0,0.0,0.0,0.0
2,3.0,2014-01-06,4.3,1.00,2014-01-07,0.0,100.0,0.77,5.0,0.0,0.0,0.0,1.0,0.0
3,9.0,2014-01-10,4.6,1.14,2014-06-29,20.0,80.0,2.36,4.9,1.0,1.0,0.0,1.0,1.0
4,14.0,2014-01-27,4.4,1.19,2014-03-15,11.8,82.4,3.13,4.9,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,0.0,2014-01-25,5.0,1.00,2014-06-05,0.0,100.0,5.63,4.2,1.0,1.0,0.0,1.0,0.0
49996,1.0,2014-01-24,5.0,1.00,2014-01-25,0.0,0.0,0.00,4.0,0.0,0.0,0.0,1.0,0.0
49997,0.0,2014-01-31,5.0,1.00,2014-05-22,0.0,100.0,3.86,5.0,0.0,0.0,1.0,0.0,1.0
49998,2.0,2014-01-14,3.0,1.00,2014-01-15,0.0,100.0,4.58,3.5,0.0,0.0,0.0,1.0,0.0


In [ ]:
df_ohe = df_ohe.drop(['signup_date','last_trip_date'],axis=1)

In [88]:
df_ohe

,trips_in_first_30_days,avg_rating_of_driver,avg_surge,surge_pct,weekday_pct,avg_dist,avg_rating_by_driver,active_users,city_1,city_2,phone_1,ultimate_black_user_1
0,4.0,4.7,1.10,15.4,46.2,3.67,5.0,1.0,1.0,0.0,1.0,1.0
1,0.0,5.0,1.00,0.0,50.0,8.26,5.0,0.0,0.0,0.0,0.0,0.0
2,3.0,4.3,1.00,0.0,100.0,0.77,5.0,0.0,0.0,0.0,1.0,0.0
3,9.0,4.6,1.14,20.0,80.0,2.36,4.9,1.0,1.0,0.0,1.0,1.0
4,14.0,4.4,1.19,11.8,82.4,3.13,4.9,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
49995,0.0,5.0,1.00,0.0,100.0,5.63,4.2,1.0,1.0,0.0,1.0,0.0
49996,1.0,5.0,1.00,0.0,0.0,0.00,4.0,0.0,0.0,0.0,1.0,0.0
49997,0.0,5.0,1.00,0.0,100.0,3.86,5.0,0.0,0.0,1.0,0.0,1.0
49998,2.0,3.0,1.00,0.0,100.0,4.58,3.5,0.0,0.0,0.0,1.0,0.0


___

### Train/Test/Split

In [85]:
X = df_ohe.drop(['active_users'], axis = 1)
y = df_ohe['active_users']

SEED = 42
TS = 0.25

# Create training and test sets
X_train, X_test, y_train, y_test = \
    train_test_split(X, y, test_size = TS, random_state=SEED, stratify=y)

print (X_train.shape)
print (y_train.shape)
print (X_test.shape)
print (y_test.shape)

(37500, 11)
(37500,)
(12500, 11)
(12500,)


## Random Forest

In [86]:
MODEL_PARAMS = {
    "n_estimators": 100,
    "criterion": 'gini',
    "max_features":'sqrt',
    "random_state":SEED
}

rf = RandomForestClassifier(**MODEL_PARAMS)
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)
y_pred_proba = rf.predict_proba(X_test)[:,1]



rf_score = round(rf.score(X_train, y_train) * 100, 2)
rf_test_score = round(rf.score(X_test, y_test) * 100, 2)

print('Random Forest Training Score:', rf_score)
print('Random Forest Test Score:', rf_test_score)


#Checking performance on our model with ROC Score.
rf_roc_score = round(roc_auc_score(y_test,y_pred_proba)*100,3)
print("ROC Score:", rf_roc_score)

RandomForestClassifier(max_features='sqrt', random_state=42)

Random Forest Training Score: 99.59
Random Forest Test Score: 76.06
ROC Score: 82.258


In [89]:
MODEL_PARAMS = {
    "solver":'lbfgs',
    "penalty":'l2',
    "n_jobs":None,
    "verbose":0,
    #"max_iter":200  
}

# Create logistic regression model
logreg = LogisticRegression(**MODEL_PARAMS)

# Train the model using the training sets
logreg.fit(X_train, y_train)

# Make predictions using the testing set
y_pred = logreg.predict(X_test)
y_pred_proba = logreg.predict_proba(X_test)[:,1]

# Compute and print the confusion matrix and classification report
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

logreg_score = round(logreg.score(X_train, y_train) * 100, 2)
logreg_test_score = round(logreg.score(X_test, y_test) * 100, 2)

print('Logistic Regression Training Score: \n', logreg_score)
print('Logistic Regression Test Score: \n', logreg_test_score)

print('Accuracy: \n', accuracy_score(y_test,y_pred))

#Checking performance on our model with ROC Score.
logreg_roc_score = round(roc_auc_score(y_test, y_pred_proba)*100,3)
print("ROC Score:", logreg_roc_score)

LogisticRegression()

[[6614 1185]
 [2388 2313]]
              precision    recall  f1-score   support

         0.0       0.73      0.85      0.79      7799
         1.0       0.66      0.49      0.56      4701

    accuracy                           0.71     12500
   macro avg       0.70      0.67      0.68     12500
weighted avg       0.71      0.71      0.70     12500

Logistic Regression Training Score: 
 71.86
Logistic Regression Test Score: 
 71.42
Accuracy: 
 0.71416
ROC Score: 75.326
